In [25]:
import pandas as pd
import numpy as np
import import_ipynb
import sys
import math 
import seaborn as sea
import datetime as dt
import Fuzzy_Prediction_Beta as fz
from datetime import timedelta
from operator import itemgetter
from functools import partial
url = "in.csv"

In [26]:
dataset = pd.read_csv(url, error_bad_lines=False)

In [27]:
selected = dataset[['cod_location','date_id','total_cases']]
selected.head()

,cod_location,date_id,total_cases
0,0,20191231,0.0
1,0,20200101,0.0
2,0,20200102,0.0
3,0,20200103,0.0
4,0,20200104,0.0


In [28]:
selected = selected.groupby('cod_location').tail(5)

In [29]:
selected.head()
#Input na rede neural
selected.to_csv('input.csv')

In [30]:
def aver(input):
    medias = input.groupby(['cod_location'])['total_cases'].mean()
    return medias

In [31]:
def diff(input):
    difference = input.groupby('cod_location')['total_cases'].apply(lambda x: x.tail(1).values[0] - x.head(1).values[0])
    return difference

In [32]:
def first(input):
    first = selected.groupby('cod_location')['total_cases'].apply(lambda x: x.head(1).values[0])
    return first

In [33]:
def last(input):
    last = selected.groupby('cod_location')['total_cases'].apply(lambda x: x.tail(1).values[0])
    return last

In [34]:
def fuzzifyErrorPos(erro):
    return fz.triangulo(erro, [0, 5, 5])

In [35]:
def fuzzifyErrorZ(erro):
    return fz.triangulo(erro, [-5, 0, 5])

In [36]:
def fuzzifyErrorNeg(erro):
    return fz.triangulo(erro, [-5,-5,0])

In [37]:
def fuzzifyErrorDotPos(erroD):
    return fz.trapezio(erroD,[1,1.5,5,5])

In [38]:
def fuzzifyErrorDotZ(erroD):
    return fz.triangulo(erroD,[-2,0,2])

In [39]:
def fuzzifyErrorDotNeg(erroD):
    return fz.trapezio(erroD,[-5,-5,-1.5,-1])

In [40]:
def agrega(regras, bot, mid, top):
    result = [0] * 200
    for regra in range(len(regras)):
        for i in range(200):
            if regras[regra][0]>0 and i < 95:
                result[i] = min(regras[regra][0], bot[i])
            if regras[regra][1]>0 and i > 90 and i<110:
                result[i] = min(regras[regra][1], bot[i])
            if regras[regra][2]>0 and i > 105 and i<200:
                result[i] = min(regras[regra][2], bot[i])
    return result

In [41]:
def OutputDecr():
    return fz.getTrapPlot(0, 200, [0, 0, 30, 95], "left")

In [42]:
def OutputPlat():
    return fz.getTriPlot(0, 200, [90, 100, 110])

In [43]:
def OutputCres():
    return fz.getTrapPlot(0, 200, [105, 170, 200, 200], "right")

In [44]:
def verifRegras(erro, erroD):
    regras = [[0] * 3 for i in range(3)]
    
    fuzzifyErrorNega = fuzzifyErrorNeg(erro)
    fuzzifyErrorZe = fuzzifyErrorZ(erro)
    fuzzifyErrorPosi= fuzzifyErrorPos(erro)
    
    fuzzifyErrorDotNega = fuzzifyErrorDotNeg(erroD)
    fuzzifyErrorDotZe = fuzzifyErrorDotZ(erroD)
    fuzzifyErrorDotPosi = fuzzifyErrorDotPos(erroD)
    
    regras[0][0] = min(fuzzifyErrorNega,fuzzifyErrorDotNega)
    regras[0][1] = min(fuzzifyErrorZe,fuzzifyErrorDotNega)
    regras[0][2] = min(fuzzifyErrorPosi,fuzzifyErrorDotNega)
    regras[1][0] = min(fuzzifyErrorNega,fuzzifyErrorDotZe)
    regras[1][1] = min(fuzzifyErrorZe,fuzzifyErrorDotZe)
    regras[1][2] = min(fuzzifyErrorPosi,fuzzifyErrorDotZe)
    regras[2][0] = min(fuzzifyErrorNega,fuzzifyErrorDotPosi)
    regras[2][1] = min(fuzzifyErrorZe,fuzzifyErrorDotPosi)
    regras[2][2] = min(fuzzifyErrorPosi,fuzzifyErrorDotPosi)
    
    return regras

In [45]:
media = aver(selected)
diffe = diff(selected)
first = first(selected)
target = media - (media * 28) / 100

In [46]:
prevError = target - first
currError = target - diffe

In [47]:
error = 1.5
#currError
dervError = 2.75
#first - diffe

In [48]:
##
Regras = verifRegras(error, dervError)
vA = agrega(Regras, OutputDecr(), OutputPlat(), OutputCres())
centroide = fz.getCentroid(vA)

print(error)
print(dervError)
print(centroide)
##

1.5
2.75
92.0
